In [97]:
import pandas as pd
import numpy as np
import nltk
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import WordNetLemmatizer
import gensim
from nltk.corpus import brown
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import re
from keras.preprocessing.text import Tokenizer
import codecs
import csv


In [ ]:
# Eenmalig downloaden
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('brown')

# Voorkomen onterechte warnings
pd.options.mode.chained_assignment = None

In [60]:
# Functie voor lemmatizing
"""
"Lemmatization, unlike Stemming, reduces the inflected words properly ensuring that the root word belongs to the language."
Lemmatization is waarschijnlijk beter dan stemming in ons project omdat de betekenis van de woorden erg belangrijk is 
"""

def cleanSentence(text):
    
    text = str(text).lower()
    
    # Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e ?-? ?mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)
    
    # tokenize
    words=text.split()
    
    # lemmatize
    wordnet_lemmatizer = WordNetLemmatizer()
    lem_sentence=[]
    for word in words:   
        lem_sentence.append(wordnet_lemmatizer.lemmatize(word, pos="v"))
        lem_sentence.append(" ")
    
    return "".join(lem_sentence)

In [88]:
# Toepassen van lemmatization functie op Train data

train_data = pd.read_csv('train_data.csv')

train_data['q1_clean'] = train_data['question1'].apply(cleanSentence)
train_data['q2_clean'] = train_data['question2'].apply(cleanSentence)


In [85]:
# The Tokenizer stores everything in the word_index during fit_on_texts
tokenizer = Tokenizer(num_words=200000)

tokenizer.fit_on_texts(train_data['q1_clean']+
                       train_data['q2_clean']+
                       test_data['q1_clean']+
                       test_data['q2_clean'])

word_index = tokenizer.word_index
print('%s unique words' % len(word_index))

AttributeError: 'float' object has no attribute 'lower'

#### Word2Vec

In [ ]:
# https://medium.com/@mishra.thedeepak/word2vec-in-minutes-gensim-nlp-python-6940f4e00980

# create trained model
sentences = brown.sents()
model = gensim.models.Word2Vec(sentences, min_count=1)
print('---- SAVE BROWN CORPUS MODEL ----')
model.save('brown_model')

# create brown_model
model = gensim.models.Word2Vec.load('brown_model')

# find similarity of words
print(model.similarity('woman', 'man'))

#### Doc2vec

In [ ]:
# https://medium.com/@mishra.thedeepak/doc2vec-simple-implementation-example-df2afbbfbad5
# https://datascience.stackexchange.com/questions/23969/sentence-similarity-prediction

# create model

tagged_data = [TaggedDocument(words=word_tokenize(_d.lower()), tags=[str(i)]) for i, _d in enumerate(question1)]

max_epochs = 100
vec_size = 20
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1)

model.build_vocab(tagged_data)

for epoch in range(max_epochs):
    print('iteration {0}'.format(epoch))
    model.train(tagged_data,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

In [12]:
from gensim.models.doc2vec import Doc2Vec

model= Doc2Vec.load("d2v.model")
#to find the vector of a document which is not in training data
test_data = word_tokenize("I love chatbots".lower())
v1 = model.infer_vector(test_data)
print("V1_infer", v1)

# to find most similar doc using tags
similar_doc = model.docvecs.most_similar('1')
print(similar_doc)

# to find vector of doc in training data using tags or in other words, printing the vector of document at index 1 in training data
print(model.docvecs['1'])

FileNotFoundError: [Errno 2] No such file or directory: 'd2v.model'

In [ ]:
from gensim.models.doc2vec import Doc2Vec

model = Doc2Vec.load("d2v.model")

question1 = train_data_head[['qid1', 'question1_stem']].copy()
print(question1)
question2 = train_data_head[['qid2', 'question2_stem']].copy()
print(question2)

# get similarity of two rows


#### Word index

#### Embedding matrix

#### Sequences

#### Validation split

#### Model structure

#### Train model

#### Submission